In [1]:
import io
from datetime import timedelta
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from qiskit import QuantumCircuit
from qiskit.circuit.library import (
    PauliFeatureMap,
    RealAmplitudes,
    ZFeatureMap,
    ZZFeatureMap,
    n_local,
    z_feature_map,
    zz_feature_map,
)
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit_algorithms.optimizers import Optimizer
from qiskit_machine_learning.algorithms import QSVC, VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.optimizers import COBYLA, CRS, SPSA
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.utils import algorithm_globals
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from ucimlrepo import fetch_ucirepo
from utils import RANDOM_SEED, display_confussion_matrix, get_dataset

# Desarrolla un QSVM

En este notebook probamos diferentes configuraciones de QSVMx parapoder elegir la más óptima.

Al igual que siempre, fijamos la semilla de los números aleatorios para que los resultados sean reproducibles.

In [2]:
algorithm_globals.random_seed = RANDOM_SEED

## Preprocesado del dataset

Utilizamos una función `get_dataset` del módulo `utilis.py` que hemos definido. El funcionamiento de esta función es exactamente el mismo que el de la sección de preprocesado del notebook [practica final](practica_final.ipynb).

In [3]:
X_train, y_train, X_test, y_test = get_dataset()

## Definición de funciones auxiliares

A continuación probamos diferentes clasificadores, los cuales se diferencian en el `feature_map`.

In [4]:
def train_QSVC(feature_map, seed: int = RANDOM_SEED) -> QSVC:
    sampler = Sampler(seed=seed)
    fidelity = ComputeUncompute(sampler=sampler)
    kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    qsvc = QSVC(quantum_kernel=kernel)
    return _fit(qsvc)


def _fit(svc: SVC) -> SVC:
    global X_train, y_train

    start = time()
    svc.fit(X_train, y_train.values.ravel())
    total_time = time() - start

    if total_time < 10:
        print(f"Tiempo total de entrenamiento: {total_time * 1000:.2} ms")
    else:
        print(f"Tiempo total de entrenamiento: {timedelta(seconds=total_time)}")

    return svc


class ExperimentLogger:
    def __init__(self):
        self._results = []

    def log(
        self,
        feature_map_name,
        classifier,
        X_train,
        y_train,
        X_test,
        y_test,
    ):
        y_pred_train = classifier.predict(X_train)
        acc_train = accuracy_score(y_train, y_pred_train)
        prec_train = precision_score(y_train, y_pred_train, zero_division=0)

        y_pred_test = classifier.predict(X_test)
        acc_test = accuracy_score(y_test, y_pred_test)
        prec_test = precision_score(y_test, y_pred_test, zero_division=0)

        self._results.append(
            {
                "Feature Map": feature_map_name,
                "Accuracy (train)": acc_train,
                "Precision (train)": prec_train,
                "Accuracy (test)": acc_test,
                "Precision (test)": prec_test,
            }
        )

        print(f"--- Resultados para {feature_map_name} ---")
        print(f"Train - Accuracy: {acc_train:.4f}, Precision: {prec_train:.4f}")
        print(f"Test  - Accuracy: {acc_test:.4f}, Precision: {prec_test:.4f}")

        cm = confusion_matrix(y_test, y_pred_test)
        display_confussion_matrix(cm, title=f"Matriz de confusión (Test)")

    @property
    def results_df(self):
        return pd.DataFrame(self._results)

In [5]:
experiment_logger = ExperimentLogger

In [6]:
number_of_features = X_train.shape[1]

## Probar diferentes modelos

In [ ]:
fmap_name = "z_feature_map"
feature_map = z_feature_map(number_of_features)
qsvc = train_QSVC(feature_map)
experiment_logger.log(fmap_name, qsvc, X_train, y_train, X_test, y_test)

In [ ]:
fmap_name = "zz_feature_map"
feature_map = zz_feature_map(number_of_features)
qsvc = train_QSVC(feature_map)
experiment_logger.log(fmap_name, qsvc, X_train, y_train, X_test, y_test)

In [ ]:
fmap_name = "PauliFeatureMap"
feature_map = PauliFeatureMap(number_of_features).decompose()
qsvc = train_QSVC(feature_map)
experiment_logger.log(fmap_name, qsvc, X_train, y_train, X_test, y_test)

## Resumen de Resultados Finales

In [ ]:
display(experiment_logger.results_df)